In [2]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.schema import Document as LlamaDocument
from langchain.docstore.document import Document

In [3]:
loader = PDFPlumberLoader(r"D:\Petramount\Courses\AI\LLM\Basic_Home_Remedies.pdf")
docs = loader.load()

In [4]:
llama_docs = [LlamaDocument(text=doc.page_content) for doc in docs]

In [5]:
hf_embedding = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
node_parser = SemanticSplitterNodeParser(embed_model=hf_embedding)
nodes = node_parser.get_nodes_from_documents(llama_docs)

c:\Users\deepa\.conda\envs\rag\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\deepa\AppData\Local\llama_index\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [14]:
# documents = [Document(page_content=node.text) for node in nodes]
documents = [Document(page_content=node.text, metadata={"source": f"chunk_{i}"}) for i, node in enumerate(nodes)]


In [15]:
embedder = HuggingFaceEmbeddings()


vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

llm = ChatOpenAI(openai_api_base = "http://localhost:1234/v1", openai_api_key = "lm_studio", model = "llama-3.2-1b-instruct")



In [17]:
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate

prompt = """
1. You are an helpful assitant 
2. Use the following pieces of context to answer the question at the end.
3. Answer only by using the context and articulate it better
4. Keep the answer crisp and limited to 3,4 sentences.

Context: {context}

Question: {question}

Answer to the question:"""


QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                  llm=llm, 
                  prompt=QA_CHAIN_PROMPT, 
                  callbacks=None, 
                  verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None,
              )

qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=True,
                  retriever=retriever,
                  return_source_documents=True,
              )

In [19]:
print(qa("remedy for flu")["result"])



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. You are an helpful assitant 
2. Use the following pieces of context to answer the question at the end.
3. Answer only by using the context and articulate it better
4. Keep the answer crisp and limited to 3,4 sentences.

Context: Context:
content:Basic Home Remedies & Healthy Lifestyle
Guide
1. Introduction
Home remedies have been used for centuries to treat common ailments using natural ingredients.
Combined with a healthy lifestyle, these remedies can support overall well-being and disease
prevention.
2. Home Remedies for Common Ailments
2.1 Cold & Flu
 Ginger Tea: Boil ginger slices in water, add honey and lemon. Helps relieve congestion
and sore throat.
 Turmeric Milk: Mix 1 tsp turmeric in warm milk. Acts as an anti-inflammatory and
boosts immunity.
 Steam Inhalation: Boil water, add eucalyptus oil, and inhale steam to clear nasal
congestion.
2.2 Cough
 Honey & Lemon: Mix 1 tbs